# requirements.txt

In [1]:
!pip install ultralytics
!pip install torch==1.13.0+cu116 torchvision==0.14.0+cu116 torchaudio==0.13.0 --extra-index-url https://download.pytorch.org/whl/cu116
!pip install numpy
!pip install pandas
!pip install Pillow
!pip install pyYAML
!pip install requests
!pip install tqdm
!pip install sahi

  Using cached ultralytics-8.0.110-py3-none-any.whl (592 kB)
  Using cached matplotlib-3.7.1-cp38-cp38-win_amd64.whl (7.6 MB)
  Using cached opencv_python-4.7.0.72-cp37-abi3-win_amd64.whl (38.2 MB)
  Using cached Pillow-9.5.0-cp38-cp38-win_amd64.whl (2.5 MB)
  Using cached PyYAML-6.0-cp38-cp38-win_amd64.whl (155 kB)
  Using cached seaborn-0.12.2-py3-none-any.whl (293 kB)
  Using cached pandas-2.0.2-cp38-cp38-win_amd64.whl (10.8 MB)
  Using cached torchvision-0.15.2-cp38-cp38-win_amd64.whl (1.2 MB)
  Using cached scipy-1.10.1-cp38-cp38-win_amd64.whl (42.2 MB)
  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
  Using cached torch-2.0.1-cp38-cp38-win_amd64.whl (172.4 MB)
  Using cached requests-2.31.0-py3-none-any.whl (62 kB)
  Using cached numpy-1.24.3-cp38-cp38-win_amd64.whl (14.9 MB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Using cached importlib_resources-5.12.0-py3-none-any.whl (36 kB)
  Using cached pyparsing-3.0.9-py3-none-any.whl (98 kB)
  Using cached contourpy-

# convert coco to yolo

In [46]:
import os
import json

In [28]:
with open("../datasets/sample_dataset/train/_annotations.coco.json", "r") as train_json:
    train_dataset = json.load(train_json)
    train_json.close()

with open("../datasets/sample_dataset/valid/_annotations.coco.json", "r") as valid_json:
    valid_dataset = json.load(valid_json)
    valid_json.close()

with open("../datasets/sample_dataset/test/_annotations.coco.json", "r") as test_json:
    test_dataset = json.load(test_json)
    test_json.close()

In [50]:
def json2yolo(dataset, save_path):
    os.makedirs(save_path, exist_ok=True)

    categories = [category["name"] for category in dataset["categories"]]
    save_categories_name = os.path.join(save_path, "classes.txt")
    lines = ""
    for category in categories:
        lines += f"{category}\n"
    
    with open(save_categories_name, "w") as f:
        f.writelines(lines)
        f.close()

    for img, anno in zip(dataset["images"], dataset["annotations"]):
        lines = ""
        file_name = img["file_name"]
        save_name = os.path.join(save_path, file_name).replace(".jpg", ".txt")
        height = img["height"]
        width = img["width"]
        category = anno["category_id"]
        bbox = anno["bbox"]
        x1, y1, x2, y2 = bbox[0], bbox[1], bbox[0]+bbox[2], bbox[1]+bbox[3]
        
        x = (x1+x2)/2/width
        y = (y1+y2)/2/height
        w = (x2-x1)/width
        h = (y2-y1)/height
        
        lines += f"{category} {round(x,6)} {round(y,6)} {round(w,6)} {round(h,6)}\n"
        
        with open(save_name, "w") as f:
            f.writelines(lines)
            f.close()

In [51]:
json2yolo(train_dataset, "../datasets/sample_dataset/train/labels")
json2yolo(valid_dataset, "../datasets/sample_dataset/valid/labels")
json2yolo(test_dataset, "../datasets/sample_dataset/test/labels")